In [5]:
import numpy as np
import math
import csv
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from tkinter import *
from tkinter import filedialog

#Establish plotly settings
plotly.tools.set_credentials_file(username='e@demonpore.com', api_key='7HOIUl4yTmgPyY5AA2aI')
#plotly.tools.set_config_file(world_readable=False, sharing='private')

#Get the data input file
root = Tk()
root.withdraw()
root.update()
filename = filedialog.askopenfilename(initialdir = "C:\\Users\\User\\Desktop\\Demonpore\\Data",
                                        title = "Select Data File",
                                        filetypes = (("Excel csv","*.csv"),("all files","*.*")))
root.destroy()

csvReader = csv.reader(open(filename))

current = []
voltage = []

for row in csvReader:
    current.append(float(row[0]))
    voltage.append(float(row[1].strip(".")))

#Set up parameters
Fs = 1250.0;
Ts = 1.0/Fs;
n = len(current)
t = np.arange(0,n/Fs,Ts)

k = np.arange(n)
T = n/Fs
frq = k/T
frq = frq[range(n//2)]

Y = np.fft.fft(current)/n
Y = Y[range(n//2)]

#Isolate DC and 60 Cycle components
DCOffset = Y[0]
ACNoise = 0

#General loop through data to either filter or isolate artifacts
for i in range(n//2):
    if (frq[i] >= 58 and frq[i] <= 62):
        if abs(Y[i]) >= ACNoise:
            ACNoise = abs(Y[i])
            ACFreq = frq[i]
#    if abs(Y[i]).real >= 0.01:
#        print("{:.2f}".format(frq[i]),"",abs(Y[i]).real)
#        Y[i] = 0

print("\nDC Offset = {:.2f}".format(DCOffset.real), "nA")
print("60 Hz noise amplitude = {:.4f}".format(ACNoise.real),
     " Centered at {:.2f}".format(ACFreq), "Hz")

plotly.offline.init_notebook_mode(connected=True)

#Plot raw data
trace1 = go.Scattergl(x=t,
                    y=current,
                    line=dict(color=('rgb(0,63,127)'),width=1),
                    name='Current')
trace2 = go.Scattergl(x=t,
                    y=voltage,
                    line=dict(color=('rgb(255,0,0)'), width=1),
                    name='Potential',
                    yaxis='y2')
data = [trace1, trace2]
#plot_url = py.plot(data, filename=filename + '_Data')
plotly.offline.plot({"data": data,
                    "layout": go.Layout(title="Acquired Data",
                                        xaxis=dict(title='Time (s)',
                                        titlefont=dict(family='Courier New, monospace',
                                                       size=16,
                                                       color='rgb(55,55,55)')
                                        ),
                                        yaxis=dict(title='Current (nA)',
                                                    titlefont=dict(family='Courier New, monospace',
                                                    size=16,
                                                    color='rgb(0,63,127)')
                                         ),
                                        yaxis2=dict(title='Potential (mV)',
                                                    titlefont=dict(family='Courier New, monospace',
                                                    size=16,
                                                    color='rgb(255,0,0)'),
                                                    overlaying='y',
                                                    side='right'
                                 )
                      )
})

#Plot DFT
#plot_url = py.plot(data, filename=filename + '_DFT')
plotly.offline.plot({"data": [go.Scattergl(x=frq, y=abs(Y).real,
                                          line=dict(color=('rgb(255,0,127)'),
                                                    width=1))],
    "layout": go.Layout(title="DFT",
                       xaxis=dict(title='Spectrum (Hz)',
                                 titlefont=dict(family='Courier New, monospace',
                                               size=16,
                                               color='rgb(55,55,55)')
                                 ),
                       yaxis=dict(title='Amplitude',
                                 titlefont=dict(family='Courier New, monospace',
                                                size=16,
                                                color='rgb(55,55,55)')
                                 )
                    )
})


DC Offset = 16.86 nA
60 Hz noise amplitude = 0.0241  Centered at 58.38 Hz


'file://C:\\Users\\User\\temp-plot.html'